<h1><div align='center'><font size='6' color='brick'> Linear Regression</font></div></h1>
<br>

## Importing packages

In [1]:
import pandas as pd 
from io import StringIO 
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
import statsmodels.formula.api as smf
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

## Loading data

This is an old data set of home prices.

In [2]:

housing = pd.read_csv("housing.csv") 
housing.head()

,crim,zn,river,rm,ptratio,medv
0,0.00632,18.0,0,6.575,15.3,24.0
1,0.02731,0.0,0,6.421,17.8,21.6
2,0.02729,0.0,0,7.185,17.8,34.7
3,0.03237,0.0,0,6.998,18.7,33.4
4,0.06905,0.0,0,7.147,18.7,36.2


## EDA


In [3]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   river    506 non-null    int64  
 3   rm       506 non-null    float64
 4   ptratio  506 non-null    float64
 5   medv     506 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 23.8 KB


There are no categorical variables. Next, we'll check for missing values.


In [4]:
housing.isnull().sum()

crim       0
zn         0
river      0
rm         0
ptratio    0
medv       0
dtype: int64

I have purposefully chosen a clean and very simple data set so that we focus more on the algorithms. Ordinarily, a lot more EDA would be needed - to visualize the distribution of each feature, identify outliers and more before we start feature engineering.

## OLS - Ordinary Least Squares method

### Simple Linear Regression

We will start by using the smf.ols() function to fit a simple linear regression model, with medv as the response and crim as the predictor.

In [5]:
result1 = smf.ols('medv ~ crim', data=housing).fit()

In [6]:
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     89.49
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           1.17e-19
Time:                        02:04:19   Log-Likelihood:                -1798.9
No. Observations:                 506   AIC:                             3602.
Df Residuals:                     504   BIC:                             3610.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.0331      0.409     58.740      0.000      23.229      24.837
crim          -0.4152      0.044     -9.460      0.000      -0.501      -0.329
==============================================================================
Omnibus:                      139.832   Durbin-Watson:                   0.713
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              295.404
Skew:                           1.490   Prob(JB):                     7.14e-65
Kurtosis:                       5.264   Cond. No.                         10.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Multiple Linear regression

In order to fit a multiple linear regression model using least squares, we again use the smf.ols() function. The syntax smf.ols('y ~ x1+x2+x3', data) is used to fit a model with three predic- tors, x1, x2,and x3. The summary() function now outputs the regression coefficients for all the predictors.

In [7]:

result2 = smf.ols('medv ~ crim+rm', data=housing).fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     297.6
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           5.22e-86
Time:                        02:04:19   Log-Likelihood:                -1642.7
No. Observations:                 506   AIC:                             3291.
Df Residuals:                     503   BIC:                             3304.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -29.2447      2.588    -11.300      0.000     -34.330     -24.160
crim          -0.2649      0.033     -8.011      0.000      -0.330      -0.200
rm             8.3911      0.405     20.726      0.000       7.596       9.186
==============================================================================
Omnibus:                      172.412   Durbin-Watson:                   0.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1047.536
Skew:                           1.349   Prob(JB):                    3.39e-228
Kurtosis:                       9.512   Cond. No.                         92.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The housing data set contains 5 covariates so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors. Instead, we can use the following short-hand:

In [10]:
housing.columns[:-1]

Index(['crim', 'zn', 'river', 'rm', 'ptratio'], dtype='object')

In [11]:
'+'.join(housing.columns[:-1])

'crim+zn+river+rm+ptratio'

In [12]:
string_cols = ' + '.join(housing.columns[:-1])
result3 = smf.ols('medv ~ {}'.format(string_cols), data=housing).fit() 
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.605
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     153.3
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           1.76e-98
Time:                        02:07:35   Log-Likelihood:                -1605.1
No. Observations:                 506   AIC:                             3222.
Df Residuals:                     500   BIC:                             3248.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.8294      4.014     -1.203      0.230     -12.716       3.057
crim          -0.1981      0.032     -6.237      0.000      -0.260      -0.136
zn             0.0277      0.012      2.230      0.026       0.003       0.052
river          3.3018      1.033      3.196      0.001       1.272       5.332
rm             7.1443      0.405     17.627      0.000       6.348       7.941
ptratio       -0.9409      0.138     -6.800      0.000      -1.213      -0.669
==============================================================================
Omnibus:                      220.975   Durbin-Watson:                   0.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1796.516
Skew:                           1.703   Prob(JB):                         0.00
Kurtosis:                      11.580   Cond. No.                         436.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

What if we would like to perform a regression using all of the variables but one? For example, in the above regression output, zn has the highest p-value (still small though). The following syntax results in a regression using all predictors except zn.

In [14]:
 string_cols = ' + '.join(housing.columns[:-1].difference(['zn'])) 
result4 = smf.ols('medv ~ {}'.format(string_cols), data=housing).fit() 
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     188.9
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           1.42e-98
Time:                        02:09:02   Log-Likelihood:                -1607.6
No. Observations:                 506   AIC:                             3225.
Df Residuals:                     501   BIC:                             3246.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.8668      4.007     -0.965      0.335     -11.739       4.005
crim          -0.2035      0.032     -6.402      0.000      -0.266      -0.141
ptratio       -1.0345      0.132     -7.816      0.000      -1.295      -0.774
river          3.0411      1.031      2.951      0.003       1.016       5.066
rm             7.3223      0.399     18.354      0.000       6.538       8.106
==============================================================================
Omnibus:                      215.939   Durbin-Watson:                   0.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1751.602
Skew:                           1.655   Prob(JB):                         0.00
Kurtosis:                      11.493   Cond. No.                         311.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Here, we use difference to exclude zn. The function difference() returns a set that is the difference between two sets. For example, if A = {100, 60} and B = {60, 20}. Then A.difference(B) = {100} and B.difference(A) = {20}.

In [15]:
string_cols = ' + '.join(housing.columns[:-1].difference(['zn']))
print(string_cols)

crim + ptratio + river + rm


## Interaction terms